In [1]:
training_path = '/kaggle/input/photo-reconstruction/Dataset/Training_Data'
testing_path = '/kaggle/input/photo-reconstruction/Dataset/Testing_Data'

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
# !pip install torch==1.9.0 torchvision==0.10.0 --extra-index-url https://download.pytorch.org/whl/cu102
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor, Compose
from torch.utils.data import Dataset



In [3]:
# Preprocessing the Dataset
class Process_Dataset(Dataset):
    def __init__(self, masked_directory, unmasked_directory):
        self.transforms = Compose([ToTensor()]) # Defining transforms
        self.unmasked_directory = unmasked_directory
        self.masked_directory = masked_directory
        self.filenames = os.listdir(masked_directory)
        self.filenames.remove('masked_info.csv') # removing the masked_info.csv file
        self.masked_info = os.path.join(self.masked_directory, 'masked_info.csv')
    
    def __getitem__(self, idx):
        unmasked_path = os.path.join(self.unmasked_directory, self.filenames[idx])#getting the images
        masked_path = os.path.join(self.masked_directory, self.filenames[idx])
        unmasked_image = Image.open(unmasked_path).convert('RGB')#converting  it to RGB format
        masked_image = Image.open(masked_path).convert('RGB') 
        unmasked_tensor = self.transforms(unmasked_image) # converting it to tenspors
        masked_tensor = self.transforms(masked_image)
        return masked_tensor, unmasked_tensor

    def __len__(self):
        return len(self.filenames) # sending the number of files

In [4]:
# Creating the BiLSTM Model Class
import torch.nn as nn
class BiLSTM_model(nn.Module):
    def __init__(self, size_in, size_h, layes):
        super(BiLSTM_model, self).__init__()
        self.lstm = nn.LSTM(size_in, size_h, layes, bidirectional=True)
        self.fc = nn.Linear(size_h*2, size_in)

    def forward(self, x):
        output, dummy = self.lstm(x)
        output = self.fc(output)
        return output

In [5]:
# The Loss function for the model
class ReconstructionLoss(nn.Module):
    def __init__(self):
        super(ReconstructionLoss, self).__init__()

    def forward(self, X, Y):
        return nn.MSELoss()(X, Y)

In [6]:
# making the model
model = BiLSTM_model(size_in=256, size_h=128, layes=2)

In [7]:
# loss function declaration
loss_function = ReconstructionLoss()

In [8]:
# Setting the optimizer as Adam and learning rate 0.0008
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
from torch.utils.data import TensorDataset, ConcatDataset

# Loading the elephant train data
elephant_train_dataset = Process_Dataset(unmasked_directory=f"{training_path}/Elephant/Unmasked_Train", masked_directory=f"{training_path}/Elephant/Masked_Train")
# Loading the tiger train data
tiger_train_dataset = Process_Dataset(unmasked_directory=f"{training_path}/Tiger/Unmasked_Train", masked_directory=f"{training_path}/Tiger/Masked_Train")
# Loading the dog train data
dog_train_dataset = Process_Dataset(unmasked_directory=f"{training_path}/Dog/Unmasked_Train", masked_directory=f"{training_path}/Dog/Masked_Train")
# Loading the cat train data
cat_train_dataset = Process_Dataset(unmasked_directory=f"{training_path}/Cat/Unmasked_Train", masked_directory=f"{training_path}/Cat/Masked_Train")
# combining all the datas
train_dataset = ConcatDataset([cat_train_dataset, dog_train_dataset, elephant_train_dataset, tiger_train_dataset])

In [10]:
from torch.utils.data import DataLoader
# loading the train dataset into DataLoader
training_dataloader = DataLoader(train_dataset, shuffle=True)

In [11]:
# Running the epochs and training the model
for epoch in range(10):
    loss = 0
    for i, (masked, unmasked) in enumerate(training_dataloader):
        optimizer.zero_grad()
        output = model(masked[0])
        
        loss_one = loss_function(output, unmasked[0])
        loss += loss_one.item()
        loss_one.backward()
        optimizer.step()
    print(f"The epoch number is  {epoch} and the mean squared error loss is = {loss}")
    torch.save(model,f'/kaggle/working/model{epoch}.pth')


The epoch number is  0 and the mean squared error loss is = 129.0024037489202
The epoch number is  1 and the mean squared error loss is = 83.17612773517612
The epoch number is  2 and the mean squared error loss is = 74.64915110135917
The epoch number is  3 and the mean squared error loss is = 70.4467000720324
The epoch number is  4 and the mean squared error loss is = 67.19362693699077
The epoch number is  5 and the mean squared error loss is = 65.16148001223337
The epoch number is  6 and the mean squared error loss is = 63.697237180313095
The epoch number is  7 and the mean squared error loss is = 62.31406541145407
The epoch number is  8 and the mean squared error loss is = 60.94648845260963
The epoch number is  9 and the mean squared error loss is = 60.029848701902665


In [12]:
torch.save(model,f'/kaggle/working/model.pth')

In [13]:
def submission_data(test_path):
    test_mask_data = pd.read_csv(test_path + 'masked_info.csv').drop(['Unnamed: 0'],axis=1)
    submission = []

    for i in range(len(test_mask_data)):
        filename, y1, x1, y2, x2 = test_mask_data.loc[i,]
        print(i+1)
        masked_image = Image.open(test_path + filename).convert('RGB')
        masked_tensor = Compose([ToTensor()])(masked_image)
        with torch.no_grad():
            output = model(masked_tensor)
        for i in range(y1 , y1+75):
            for j in range(x1, x1+75):
                temp1 = filename + '_' + 'box1' + '_' +  str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', output[2][i][j].item()))#B of output and 2 of RGB
                submission.append((temp1 + '1', output[1][i][j].item()))#G of output and 1 of RGB
                submission.append((temp1 + '2', output[0][i][j].item()))#R of output and 0 of RGB

        for i in range(y2 , y2+75):
            for j in range(x2, x2+75):
                temp1 = filename + '_' + 'box2' + '_' + str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', output[2][i][j].item()))#B of output and 2 of RGB
                submission.append((temp1 + '1', output[1][i][j].item()))#G of output and 1 of RGB
                submission.append((temp1 + '2', output[0][i][j].item()))#R of output and 0 of RGB
    df = pd.DataFrame(submission, columns=['filename_box_pixel','value'])
    return df;

In [14]:
test_path = "/kaggle/input/photo-reconstruction/Dataset/Testing_Data/"
df = submission_data(test_path)
output_path = '/kaggle/working/'
df.to_csv(f"{output_path}/submission.csv", index = False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [15]:
df.head()

,filename_box_pixel,value
0,Tiger (1024).jpeg_box1_58_27_0,0.203640
1,Tiger (1024).jpeg_box1_58_27_1,0.390237
2,Tiger (1024).jpeg_box1_58_27_2,0.299468
3,Tiger (1024).jpeg_box1_58_28_0,0.205701
4,Tiger (1024).jpeg_box1_58_28_1,0.372379
